In [1]:
!pip install gradio twilio folium

import gradio as gr
from twilio.rest import Client
import json
import os
import folium
from folium.plugins import MarkerCluster

TWILIO_ACCOUNT_SID = 'ACbdd5a649c9fe8f9ac507bec078dfae5f'
TWILIO_AUTH_TOKEN = '9c85960ca817ecdbbece3748d42fe2b3'
TWILIO_PHONE_NUMBER = '+15418278057'

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

users_db = {}

SESSION_FILE = "session_store.json"

def save_session(user):
    with open(SESSION_FILE, "w") as f:
        json.dump({"user": user}, f)

def load_session():
    try:
        with open(SESSION_FILE, "r") as f:
            return json.load(f)
    except:
        return {"user": None}

def clear_session():
    with open(SESSION_FILE, "w") as f:
        json.dump({"user": None}, f)

police_stations = [
    {"name": "Central Police", "contact": "100", "location": "Main Street", "coords": [11.0168, 76.9558]},
    {"name": "North Zone", "contact": "101", "location": "North Avenue", "coords": [11.0250, 76.9500]},
    {"name": "East Control", "contact": "102", "location": "East Cross Road", "coords": [11.0300, 76.9600]}
]

def send_sms_alert(username):
    user = users_db.get(username)
    if not user:
        return "❌ User not found."

    contacts = user["contacts"]
    phone = user["phone"]
    gps_link = user.get("gps_location", "Not provided")

    message_body = (
        f"🚨 EMERGENCY ALERT 🚨\n"
        f"User: {username}\n"
        f"Phone: {phone}\n"
        f"Location: {gps_link}\n"
        f"Needs immediate assistance!"
    )

    try:
        for number in contacts:
            client.messages.create(
                body=message_body,
                from_=TWILIO_PHONE_NUMBER,
                to=number
            )
        return f"✅ Alert sent to {len(contacts)} contacts."
    except Exception as e:
        return f"❌ SMS failed: {str(e)}"

def signup(username, password, phone, contacts, gps_location):
    if not all([username, password, phone, contacts, gps_location]):
        return "⚠ All fields are required!"

    if username in users_db:
        return "⚠ Username already exists. Please login."

    contacts_list = [c.strip() for c in contacts.split(",") if c.strip()]
    users_db[username] = {
        "password": password,
        "phone": phone,
        "contacts": contacts_list,
        "gps_location": gps_location
    }

    return "✅ Signup successful! Please login now."

def login(username, password):
    user = users_db.get(username)
    if user and user["password"] == password:
        save_session(username)
        return f"✅ Logged in as *{username}*", load_session()
    return "❌ Incorrect username or password.", load_session()

def logout():
    clear_session()
    return "👋 You have been logged out.", load_session()

def emergency_alert(image, video, session):
    username = session["user"]
    if not username:
        return "⚠ You must login to trigger emergency."

    backup_dir = "resqcam_backup"
    os.makedirs(backup_dir, exist_ok=True)
    evidence = []

    if image:
        img_path = os.path.join(backup_dir, f"{username}_image.jpg")
        image.save(img_path)
        evidence.append(img_path)

    if video:
        vid_path = os.path.join(backup_dir, f"{username}_video.mp4")
        with open(vid_path, "wb") as f:
            f.write(video.read())
        evidence.append(vid_path)

    sms_status = send_sms_alert(username)

    alert_msg = f"🚨 Emergency triggered by *{username}*\n\n📤 {sms_status}"
    if evidence:
        alert_msg += f"\n📎 Evidence saved: {', '.join(evidence)}"

    alert_msg += "\n\n📍 Nearest Police Stations:\n"
    for ps in police_stations:
        alert_msg += f"🔹 {ps['name']} | 📞 {ps['contact']} | 📍 {ps['location']}\n"

    return alert_msg

def about_info():
    return (
        "### 👁 ResqCam - Smart Emergency Surveillance System\n"
        "✅ Sign up with GPS live link\n"
        "📎 Capture photo/video\n"
        "📤 Send real-time SMS alerts\n"
        "📍 Automatically include saved GPS location\n"
        "🗺 View police stations\n\n"
        "Built using Python, Gradio, and Twilio"
    )

def generate_map():
    m = folium.Map(location=[11.0168, 76.9558], zoom_start=13)
    cluster = MarkerCluster().add_to(m)
    for ps in police_stations:
        folium.Marker(
            location=ps["coords"],
            popup=f"{ps['name']} - 📞 {ps['contact']}",
            icon=folium.Icon(color='blue')
        ).add_to(cluster)
    return m._repr_html_()

with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("<h1 align='center' style='color:#3b82f6;'>🛡 ResqCam</h1><h4 align='center'>Smart Emergency Surveillance System</h4>")

    session = gr.State(load_session)

    with gr.Tab("📝 Signup"):
        gr.Markdown("### 🔐 Create Your Account")
        s_user = gr.Textbox(label="👤 Username")
        s_pass = gr.Textbox(label="🔑 Password", type="password")
        s_phone = gr.Textbox(label="📱 Phone Number (with +)")
        s_contacts = gr.Textbox(label="📞 Emergency Contacts", placeholder="Comma-separated numbers")
        s_gps = gr.Textbox(label="📍 GPS Location Link", placeholder="Paste your live GPS link (e.g., Google Maps)")
        s_btn = gr.Button("✅ Signup")
        s_result = gr.Textbox(label="Status", interactive=False)
        s_btn.click(signup, [s_user, s_pass, s_phone, s_contacts, s_gps], s_result)

    with gr.Tab("🔓 Login / Logout"):
        gr.Markdown("### 🔐 Login")
        l_user = gr.Textbox(label="👤 Username")
        l_pass = gr.Textbox(label="🔑 Password", type="password")
        l_btn = gr.Button("🔓 Login")
        l_result = gr.Textbox(label="Login Status", interactive=False)
        l_btn.click(login, [l_user, l_pass], [l_result, session])

        logout_btn = gr.Button("🚪 Logout")
        logout_out = gr.Textbox(label="Logout Status", interactive=False)
        logout_btn.click(logout, outputs=[logout_out, session])

    with gr.Tab("🚨 Trigger Emergency"):
        gr.Markdown("### 📷 Capture Evidence and Send Alert")
        image = gr.Image(label="📸 Capture Image", sources=["webcam"], type="pil")
        video = gr.Video(label="🎥 Record Video", sources=["webcam"])
        em_btn = gr.Button("🚨 Trigger Emergency")
        em_out = gr.Textbox(label="Emergency Alert Log", lines=10, interactive=False)
        em_btn.click(emergency_alert, inputs=[image, video, session], outputs=em_out)

    with gr.Tab("🗺 Police Stations Map"):
        gr.Markdown("### 🗺 Nearby Police Stations")
        gr.HTML(generate_map())

    with gr.Tab("ℹ About ResqCam"):
        gr.Markdown("### 💡 About ResqCam")
        about_btn = gr.Button("📘 Show Info")
        about_out = gr.Textbox(label="About", lines=10, interactive=False)
        about_btn.click(about_info, outputs=about_out)

app.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15f66204c759faca2e.gradio.live

This share link expires in 1 week. For free permanent hosting and 